This notebook will join application_train with credit card + instalment + previous application table. Credit card and installment are joined with previous application on SK_ID_PREV. Previous application is joined with application_train on SK_ID_CURR. 

We will join credit card and installment with previous application first, then join them with the application_train

# 1. Loading libraries and import data

In [146]:
# import libraries and suppress memory
# importing Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# define the function for reducing memory usage when importing data
def reduce_memory_usage(df):
  
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [148]:
#reading the csv file 
application_train = reduce_memory_usage(pd.read_csv("/Users/linhdo/Documents/University/University of Utah/Coursework/Spring 2025/Career Capstone/home-credit-default-risk/application_final.csv"))
credit_card = reduce_memory_usage(pd.read_csv("/Users/linhdo/Documents/University/University of Utah/Coursework/Spring 2025/Career Capstone/home-credit-default-risk/credit_card_balance.csv"))
instalment = reduce_memory_usage(pd.read_csv("/Users/linhdo/Documents/University/University of Utah/Coursework/Spring 2025/Career Capstone/home-credit-default-risk/installments_payments.csv"))
previous_df = reduce_memory_usage(pd.read_csv("/Users/linhdo/Documents/University/University of Utah/Coursework/Spring 2025/Career Capstone/home-credit-default-risk/previous_application.csv"))

Memory usage of dataframe is 17.05 MB
Memory usage after optimization is: 6.72 MB
Decreased by 60.6%
Memory usage of dataframe is 673.88 MB
Memory usage after optimization is: 289.33 MB
Decreased by 57.1%
Memory usage of dataframe is 830.41 MB
Memory usage after optimization is: 311.40 MB
Decreased by 62.5%
Memory usage of dataframe is 471.48 MB
Memory usage after optimization is: 309.01 MB
Decreased by 34.5%


# 2. Data Exploration 

## a. Credit Card

In [150]:
credit_card.head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.500000,0.0,877.500000,1700.324951,1800.000000,1800.000000,0.000000,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.000000,0.0,0.000000,2250.000000,2250.000000,2250.000000,60175.078125,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.000000,0.0,0.000000,2250.000000,2250.000000,2250.000000,26926.425781,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.000000,0.0,0.000000,11795.759766,11925.000000,11925.000000,224949.281250,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.000000,0.0,11547.000000,22924.890625,27000.000000,27000.000000,443044.406250,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0
5,2646502,380010,-7,82903.812500,270000,0.0,0.000000,0.0,0.000000,4449.104980,3825.000000,3825.000000,80519.039062,82773.312500,82773.312500,0.0,0,0.0,0.0,2.0,Active,7,0
6,1079071,171320,-6,353451.656250,585000,67500.0,67500.000000,0.0,0.000000,14684.174805,15750.000000,15750.000000,345433.875000,351881.156250,351881.156250,1.0,1,0.0,0.0,6.0,Active,0,0
7,2095912,118650,-7,47962.125000,45000,45000.0,45000.000000,0.0,0.000000,0.000000,264.690002,0.000000,44735.308594,47962.125000,47962.125000,1.0,1,0.0,0.0,51.0,Active,0,0
8,2181852,367360,-4,291543.062500,292500,90000.0,289339.437500,0.0,199339.421875,130.500000,4093.514893,4093.514893,285376.406250,286831.562500,286831.562500,3.0,8,0.0,5.0,3.0,Active,0,0
9,1235299,203885,-5,201261.187500,225000,76500.0,111026.703125,0.0,34526.699219,6338.339844,45000.000000,45000.000000,192793.281250,197224.687500,197224.687500,3.0,9,0.0,6.0,38.0,Active,0,0


In [152]:
credit_card = credit_card.sort_values('SK_ID_PREV')

# view all columns
pd.set_option('display.max_columns', None)

credit_card.head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
2159094,1000018,394447,-2,136695.421875,135000,0.0,22827.330078,0.0,22827.330078,6206.669922,7970.535156,7970.535156,132903.000000,136024.921875,136024.921875,0.0,11,0.0,11.0,4.0,Active,0,0
2559582,1000018,394447,-4,44360.503906,45000,0.0,2032.560059,0.0,2032.560059,2250.000000,3646.709961,3646.709961,43376.761719,44360.503906,44360.503906,0.0,2,0.0,2.0,2.0,Active,0,0
277653,1000018,394447,-5,40934.070312,45000,0.0,2335.500000,0.0,2335.500000,2250.000000,3900.870117,3900.870117,39576.781250,40934.070312,40934.070312,0.0,2,0.0,2.0,1.0,Active,0,0
2192275,1000018,394447,-3,113862.281250,135000,13500.0,69156.945312,0.0,55656.945312,2263.770020,3190.635010,3190.635010,108091.796875,109150.781250,109150.781250,3.0,14,0.0,11.0,3.0,Active,0,0
1375415,1000018,394447,-6,38879.144531,45000,13500.0,51042.644531,0.0,37542.644531,0.000000,9000.000000,9000.000000,37542.644531,37542.644531,37542.644531,3.0,15,0.0,12.0,0.0,Active,0,0
866526,1000030,361282,-3,100264.453125,112500,0.0,46660.500000,0.0,46660.500000,2250.000000,4849.649902,322.380005,99775.257812,100264.453125,100264.453125,0.0,4,0.0,4.0,3.0,Active,0,0
1884819,1000030,361282,-8,0.000000,45000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0,NaN,NaN,0.0,Active,0,0
1597893,1000030,361282,-2,99195.078125,112500,0.0,1849.050049,0.0,1849.050049,4527.270020,6442.694824,1094.175049,98019.093750,99195.078125,99195.078125,0.0,2,0.0,2.0,4.0,Active,0,0
1150224,1000030,361282,-1,103027.273438,112500,0.0,6550.649902,0.0,6550.649902,5348.520020,5998.589844,567.539978,101866.726562,103027.273438,103027.273438,0.0,3,0.0,3.0,5.0,Active,0,0
2192311,1000030,361282,-6,33784.738281,45000,0.0,20212.650391,0.0,20212.650391,0.000000,2567.655029,317.654999,33725.746094,33784.738281,33784.738281,0.0,8,0.0,8.0,0.0,Active,0,0


In [153]:
credit_card.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3840312 entries, 2159094 to 2575507
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   SK_ID_PREV                  int32  
 1   SK_ID_CURR                  int32  
 2   MONTHS_BALANCE              int8   
 3   AMT_BALANCE                 float32
 4   AMT_CREDIT_LIMIT_ACTUAL     int32  
 5   AMT_DRAWINGS_ATM_CURRENT    float32
 6   AMT_DRAWINGS_CURRENT        float32
 7   AMT_DRAWINGS_OTHER_CURRENT  float32
 8   AMT_DRAWINGS_POS_CURRENT    float32
 9   AMT_INST_MIN_REGULARITY     float32
 10  AMT_PAYMENT_CURRENT         float32
 11  AMT_PAYMENT_TOTAL_CURRENT   float32
 12  AMT_RECEIVABLE_PRINCIPAL    float32
 13  AMT_RECIVABLE               float32
 14  AMT_TOTAL_RECEIVABLE        float32
 15  CNT_DRAWINGS_ATM_CURRENT    float16
 16  CNT_DRAWINGS_CURRENT        int16  
 17  CNT_DRAWINGS_OTHER_CURRENT  float16
 18  CNT_DRAWINGS_POS_CURRENT    float16
 19  CNT_INSTALMENT_MATUR

In [154]:
# Define aggregation functions
agg_funcs = {
    'SK_ID_CURR': 'first',  # Since SK_ID_CURR is the same for a given SK_ID_PREV
    'MONTHS_BALANCE': lambda x: abs(x.min() - x.max()),  # Duration calculation
    'AMT_BALANCE': 'median',
    'AMT_CREDIT_LIMIT_ACTUAL': 'median',
    'AMT_DRAWINGS_CURRENT': 'median',
    'AMT_DRAWINGS_OTHER_CURRENT': 'median',
    'AMT_DRAWINGS_POS_CURRENT': 'median',
    'AMT_PAYMENT_TOTAL_CURRENT': 'median',
    'AMT_RECEIVABLE_PRINCIPAL': 'median',
    'AMT_TOTAL_RECEIVABLE': 'median',
    'CNT_INSTALMENT_MATURE_CUM': 'max',
    'SK_DPD': 'median',
    'SK_DPD_DEF': 'median'
}

# Perform aggregation
ccard_agg = credit_card.groupby('SK_ID_PREV').agg(agg_funcs).reset_index()

# Get the most recent NAME_CONTRACT_STATUS (based on max MONTHS_BALANCE)
latest_status = credit_card.loc[credit_card.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax(), 
                              ['SK_ID_PREV', 'NAME_CONTRACT_STATUS']]

# Merge with the aggregated data
ccard_agg = ccard_agg.merge(latest_status, on='SK_ID_PREV', how='left')

# Rename columns
ccard_agg.rename(columns={'MONTHS_BALANCE': 'DURATION_MONTHS'}, inplace=True)

# Rename columns to include 'MEDIAN_' prefix where applicable
ccard_agg.rename(columns={
    'AMT_BALANCE': 'MEDIAN_AMT_BALANCE',
    'AMT_CREDIT_LIMIT_ACTUAL': 'MEDIAN_AMT_CREDIT_LIMIT_ACTUAL',
    'AMT_DRAWINGS_CURRENT': 'MEDIAN_AMT_DRAWINGS_CURRENT',
    'AMT_DRAWINGS_OTHER_CURRENT': 'MEDIAN_AMT_DRAWINGS_OTHER_CURRENT',
    'AMT_DRAWINGS_POS_CURRENT': 'MEDIAN_AMT_DRAWINGS_POS_CURRENT',
    'AMT_PAYMENT_TOTAL_CURRENT': 'MEDIAN_AMT_PAYMENT_TOTAL_CURRENT',
    'AMT_RECEIVABLE_PRINCIPAL': 'MEDIAN_AMT_RECEIVABLE_PRINCIPAL',
    'AMT_TOTAL_RECEIVABLE': 'MEDIAN_AMT_TOTAL_RECEIVABLE',
    'SK_DPD': 'MEDIAN_SK_DPD',
    'SK_DPD_DEF': 'MEDIAN_SK_DPD_DEF'
}, inplace=True)

# Display result
ccard_agg.head()

,SK_ID_PREV,SK_ID_CURR,DURATION_MONTHS,MEDIAN_AMT_BALANCE,MEDIAN_AMT_CREDIT_LIMIT_ACTUAL,MEDIAN_AMT_DRAWINGS_CURRENT,MEDIAN_AMT_DRAWINGS_OTHER_CURRENT,MEDIAN_AMT_DRAWINGS_POS_CURRENT,MEDIAN_AMT_PAYMENT_TOTAL_CURRENT,MEDIAN_AMT_RECEIVABLE_PRINCIPAL,MEDIAN_AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM,MEDIAN_SK_DPD,MEDIAN_SK_DPD_DEF,NAME_CONTRACT_STATUS
0,1000018,394447,4,44360.503906,45000.0,22827.330078,0.0,22827.330078,3900.870117,43376.761719,44360.503906,4.0,0.0,0.0,Active
1,1000030,361282,7,48036.667969,78750.0,13381.650146,0.0,20212.650391,444.959991,47639.025391,48036.667969,5.0,0.0,0.0,Active
2,1000031,131335,15,2902.747559,144000.0,3382.672485,0.0,2862.000000,4105.665161,2902.747559,2902.747559,10.0,0.0,0.0,Active
3,1000035,436351,4,0.000000,225000.0,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,0.0,Active
4,1000077,181153,10,0.000000,135000.0,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,0.0,Active


In [158]:
ccard_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104307 entries, 0 to 104306
Data columns (total 15 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   SK_ID_PREV                         104307 non-null  int32  
 1   SK_ID_CURR                         104307 non-null  int32  
 2   DURATION_MONTHS                    104307 non-null  int8   
 3   MEDIAN_AMT_BALANCE                 104307 non-null  float64
 4   MEDIAN_AMT_CREDIT_LIMIT_ACTUAL     104307 non-null  float64
 5   MEDIAN_AMT_DRAWINGS_CURRENT        104307 non-null  float64
 6   MEDIAN_AMT_DRAWINGS_OTHER_CURRENT  72513 non-null   float32
 7   MEDIAN_AMT_DRAWINGS_POS_CURRENT    72513 non-null   float64
 8   MEDIAN_AMT_PAYMENT_TOTAL_CURRENT   104307 non-null  float64
 9   MEDIAN_AMT_RECEIVABLE_PRINCIPAL    104307 non-null  float64
 10  MEDIAN_AMT_TOTAL_RECEIVABLE        104307 non-null  float64
 11  CNT_INSTALMENT_MATURE_CUM          1043

In [160]:
duplicate_id_curr = ccard_agg['SK_ID_CURR'].value_counts()
duplicates = duplicate_id_curr[duplicate_id_curr > 1]

print(f"Number of duplicate SK_ID_CURR: {len(duplicates)}")
print(duplicates.head())  # Show first few duplicates

Number of duplicate SK_ID_CURR: 740
SK_ID_CURR
355767    4
120076    3
187294    3
206455    3
263421    3
Name: count, dtype: int64


## b. Instalments payments table 

In [163]:
instalment.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.359863,6948.359863
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525024,1716.525024
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000000,25425.000000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130859,24350.130859
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040039,2160.584961


In [165]:
instalment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_PREV              int32  
 1   SK_ID_CURR              int32  
 2   NUM_INSTALMENT_VERSION  float16
 3   NUM_INSTALMENT_NUMBER   int16  
 4   DAYS_INSTALMENT         float16
 5   DAYS_ENTRY_PAYMENT      float16
 6   AMT_INSTALMENT          float32
 7   AMT_PAYMENT             float32
dtypes: float16(3), float32(2), int16(1), int32(2)
memory usage: 311.4 MB


In [167]:
instalment = instalment.sort_values('SK_ID_PREV')
instalment.head(10)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
512588,1000001,158271,1.0,1,-268.0,-294.0,6404.310059,6404.310059
2159480,1000001,158271,2.0,2,-238.0,-244.0,62039.113281,62039.113281
3411021,1000002,101962,1.0,3,-1540.0,-1559.0,6264.000000,6264.000000
1214732,1000002,101962,1.0,1,-1600.0,-1611.0,6264.000000,6264.000000
1006868,1000002,101962,2.0,4,-1510.0,-1554.0,18443.564453,18443.564453
1631862,1000002,101962,1.0,2,-1570.0,-1575.0,6264.000000,6264.000000
7054246,1000003,252457,1.0,2,-64.0,-81.0,4951.350098,4951.350098
6145031,1000003,252457,1.0,1,-94.0,-108.0,4951.350098,4951.350098
4279518,1000003,252457,1.0,3,-34.0,-49.0,4951.350098,4951.350098
5197652,1000004,260094,1.0,5,-742.0,-752.0,3391.110107,3391.110107


In [168]:
instalment.isna().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
NUM_INSTALMENT_NUMBER        0
DAYS_INSTALMENT              0
DAYS_ENTRY_PAYMENT        2905
AMT_INSTALMENT               0
AMT_PAYMENT               2905
dtype: int64

In [171]:
# Add DIFF_PAY_VS_INSTAL_DAY
instalment['DIFF_PAY_VS_INSTAL_DAY'] = instalment['DAYS_INSTALMENT'] - instalment['DAYS_ENTRY_PAYMENT']

# Add PAY_ONTIME
instalment['PAY_ONTIME'] = instalment['DIFF_PAY_VS_INSTAL_DAY'].apply(
    lambda x: 'ontime' if x == 0 else ('early' if x > 0 else 'late')
)

# Add DIFF_PAY_VS_INSTAL_AMT
instalment['DIFF_PAY_VS_INSTAL_AMT'] = instalment['AMT_INSTALMENT'] - instalment['AMT_PAYMENT']

# Add ENOUGH_PAY
instalment['ENOUGH_PAY'] = instalment['DIFF_PAY_VS_INSTAL_AMT'].apply(
    lambda x: 'enough' if x == 0 else ('pay less than instalment' if x > 0 else 'pay more than instalment')
)

# Display the updated DataFrame
instalment.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,DIFF_PAY_VS_INSTAL_DAY,PAY_ONTIME,DIFF_PAY_VS_INSTAL_AMT,ENOUGH_PAY
512588,1000001,158271,1.0,1,-268.0,-294.0,6404.310059,6404.310059,26.0,early,0.0,enough
2159480,1000001,158271,2.0,2,-238.0,-244.0,62039.113281,62039.113281,6.0,early,0.0,enough
3411021,1000002,101962,1.0,3,-1540.0,-1559.0,6264.000000,6264.000000,19.0,early,0.0,enough
1214732,1000002,101962,1.0,1,-1600.0,-1611.0,6264.000000,6264.000000,11.0,early,0.0,enough
1006868,1000002,101962,2.0,4,-1510.0,-1554.0,18443.564453,18443.564453,44.0,early,0.0,enough


In [172]:
# Group by SK_ID_PREV and aggregate
instalment_agg = instalment.groupby('SK_ID_PREV').agg(
    SK_ID_CURR=('SK_ID_CURR', 'first'),
    max_num_instalment_version=('NUM_INSTALMENT_VERSION', 'max'),
    total_instalment_number=('NUM_INSTALMENT_NUMBER', 'max'),
    mode_pay_ontime=('PAY_ONTIME', lambda x: x.mode()[0] if not x.mode().empty else None),
    mode_enough_pay=('ENOUGH_PAY', lambda x: x.mode()[0] if not x.mode().empty else None)
).reset_index()

# Display the result
instalment_agg.head()

,SK_ID_PREV,SK_ID_CURR,max_num_instalment_version,total_instalment_number,mode_pay_ontime,mode_enough_pay
0,1000001,158271,2.0,2,early,enough
1,1000002,101962,2.0,4,early,enough
2,1000003,252457,1.0,3,early,enough
3,1000004,260094,2.0,7,early,enough
4,1000005,176456,1.0,10,early,enough


In [175]:
instalment_agg.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 997752 entries, 0 to 997751
Data columns (total 6 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   SK_ID_PREV                  997752 non-null  int32  
 1   SK_ID_CURR                  997752 non-null  int32  
 2   max_num_instalment_version  997752 non-null  float16
 3   total_instalment_number     997752 non-null  int16  
 4   mode_pay_ontime             997752 non-null  object 
 5   mode_enough_pay             997752 non-null  object 
dtypes: float16(1), int16(1), int32(2), object(2)
memory usage: 26.6+ MB


## c. Previous application table

Cleaning: We will follow the cleaning process on previous application table in Hung's notebook

In [182]:
previous_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1669868 entries, 0 to 1670213
Data columns (total 37 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1669868 non-null  int32  
 1   SK_ID_CURR                   1669868 non-null  int32  
 2   NAME_CONTRACT_TYPE           1669868 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float32
 4   AMT_APPLICATION              1669868 non-null  float32
 5   AMT_CREDIT                   1669867 non-null  float32
 6   AMT_DOWN_PAYMENT             1669868 non-null  float32
 7   AMT_GOODS_PRICE              1284699 non-null  float32
 8   WEEKDAY_APPR_PROCESS_START   1669868 non-null  object 
 9   HOUR_APPR_PROCESS_START      1669868 non-null  int8   
 10  FLAG_LAST_APPL_PER_CONTRACT  1669868 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       1669868 non-null  int8   
 12  RATE_DOWN_PAYMENT            1669868 non-null  

In [186]:
# Drop rows where contract type is "XNA"
previous_df = previous_df[previous_df['NAME_CONTRACT_TYPE'] != 'XNA']

# For 'NAME_CONTRACT_TYPE' being 'Cash loans' or 'Revolving loans', fill NAs with 0
previous_df.loc[previous_df['NAME_CONTRACT_TYPE'].isin(['Cash loans', 'Revolving loans']), 'AMT_APPLICATION'] = previous_df.loc[previous_df['NAME_CONTRACT_TYPE'].isin(['Cash loans', 'Revolving loans']), 'AMT_APPLICATION'].fillna(0)

# For 'NAME_CONTRACT_TYPE' being 'Consumer loans', fill missing values in 'AMT_APPLICATION' with the median
consumer_loan_median = previous_df[previous_df['NAME_CONTRACT_TYPE'] == 'Consumer loans']['AMT_APPLICATION'].median()
previous_df.loc[previous_df['NAME_CONTRACT_TYPE'] == 'Consumer loans', 'AMT_APPLICATION'] = previous_df.loc[previous_df['NAME_CONTRACT_TYPE'] == 'Consumer loans', 'AMT_APPLICATION'].fillna(consumer_loan_median)

# Fill missing values in 'NAME_TYPE_SUITE' with 'Other'
previous_df['NAME_TYPE_SUITE'].fillna('Other', inplace=True)

# Drop the 'PRODUCT_COMBINATION' column
previous_df.drop(columns=['PRODUCT_COMBINATION'], inplace=True)

# Calculate the percentage of missing values in each column
missing_percentage = previous_df.isnull().mean() * 100

# Drop columns with > 65% missing values
previous_df.drop(columns=missing_percentage[missing_percentage > 65].index, inplace=True)

In [188]:
previous_df.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430054,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,XAP,Approved,-73,Cash through the bank,XAP,Other,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615234,607500.0,679671.0,0.0,607500.0,THURSDAY,11,Y,1,0.0,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735352,112500.0,136444.5,0.0,112500.0,TUESDAY,11,Y,1,0.0,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335938,450000.0,470790.0,0.0,450000.0,MONDAY,7,Y,1,0.0,XNA,Approved,-512,Cash through the bank,XAP,Other,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.394531,337500.0,404055.0,0.0,337500.0,THURSDAY,9,Y,1,0.0,Repairs,Refused,-781,Cash through the bank,HC,Other,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
previous_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1669868 entries, 0 to 1670213
Data columns (total 34 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1669868 non-null  int32  
 1   SK_ID_CURR                   1669868 non-null  int32  
 2   NAME_CONTRACT_TYPE           1669868 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float32
 4   AMT_APPLICATION              1669868 non-null  float32
 5   AMT_CREDIT                   1669867 non-null  float32
 6   AMT_DOWN_PAYMENT             1669868 non-null  float32
 7   AMT_GOODS_PRICE              1284699 non-null  float32
 8   WEEKDAY_APPR_PROCESS_START   1669868 non-null  object 
 9   HOUR_APPR_PROCESS_START      1669868 non-null  int8   
 10  FLAG_LAST_APPL_PER_CONTRACT  1669868 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       1669868 non-null  int8   
 12  RATE_DOWN_PAYMENT            1669868 non-null  

# 3. Join credit card and instalment with previous application

In [196]:
# Perform left join with ccard_agg using SK_ID_PREV
joined_prev = previous_df.merge(ccard_agg, on="SK_ID_PREV", how="left")

# Perform left join with instalment_agg using SK_ID_PREV
joined_prev = joined_prev.merge(instalment_agg, on="SK_ID_PREV", how="left")

In [198]:
joined_prev.head()

,SK_ID_PREV,SK_ID_CURR_x,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS_x,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,SK_ID_CURR_y,DURATION_MONTHS,MEDIAN_AMT_BALANCE,MEDIAN_AMT_CREDIT_LIMIT_ACTUAL,MEDIAN_AMT_DRAWINGS_CURRENT,MEDIAN_AMT_DRAWINGS_OTHER_CURRENT,MEDIAN_AMT_DRAWINGS_POS_CURRENT,MEDIAN_AMT_PAYMENT_TOTAL_CURRENT,MEDIAN_AMT_RECEIVABLE_PRINCIPAL,MEDIAN_AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM,MEDIAN_SK_DPD,MEDIAN_SK_DPD_DEF,NAME_CONTRACT_STATUS_y,SK_ID_CURR,max_num_instalment_version,total_instalment_number,mode_pay_ontime,mode_enough_pay
0,2030495,271877,Consumer loans,1730.430054,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,XAP,Approved,-73,Cash through the bank,XAP,Other,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,365243.0,-42.0,300.0,-42.0,-37.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271877.0,2.0,1.0,ontime,enough
1,2802425,108129,Cash loans,25188.615234,607500.0,679671.0,0.0,607500.0,THURSDAY,11,Y,1,0.0,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,365243.0,-134.0,916.0,365243.0,365243.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108129.0,1.0,5.0,early,enough
2,2523466,122040,Cash loans,15060.735352,112500.0,136444.5,0.0,112500.0,TUESDAY,11,Y,1,0.0,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,365243.0,-271.0,59.0,365243.0,365243.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122040.0,1.0,9.0,early,enough
3,2819243,176158,Cash loans,47041.335938,450000.0,470790.0,0.0,450000.0,MONDAY,7,Y,1,0.0,XNA,Approved,-512,Cash through the bank,XAP,Other,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,176158.0,2.0,11.0,early,enough
4,1784265,202054,Cash loans,31924.394531,337500.0,404055.0,0.0,337500.0,THURSDAY,9,Y,1,0.0,Repairs,Refused,-781,Cash through the bank,HC,Other,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [202]:
joined_prev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669868 entries, 0 to 1669867
Data columns (total 53 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   SK_ID_PREV                         1669868 non-null  int32  
 1   SK_ID_CURR_x                       1669868 non-null  int32  
 2   NAME_CONTRACT_TYPE                 1669868 non-null  object 
 3   AMT_ANNUITY                        1297979 non-null  float32
 4   AMT_APPLICATION                    1669868 non-null  float32
 5   AMT_CREDIT                         1669867 non-null  float32
 6   AMT_DOWN_PAYMENT                   1669868 non-null  float32
 7   AMT_GOODS_PRICE                    1284699 non-null  float32
 8   WEEKDAY_APPR_PROCESS_START         1669868 non-null  object 
 9   HOUR_APPR_PROCESS_START            1669868 non-null  int8   
 10  FLAG_LAST_APPL_PER_CONTRACT        1669868 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY  

After joining, we remove the SK_ID_CURR from the credit card and instalment since we already have it in previous application.

In [208]:
# Drop the duplicate SK_ID_CURR columns
joined_prev = joined_prev.drop(columns=["SK_ID_CURR", "SK_ID_CURR_y"])

In [212]:
joined_prev = joined_prev.drop(columns=["NAME_CONTRACT_STATUS_y"])

In [214]:
joined_prev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669868 entries, 0 to 1669867
Data columns (total 50 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   SK_ID_PREV                         1669868 non-null  int32  
 1   SK_ID_CURR_x                       1669868 non-null  int32  
 2   NAME_CONTRACT_TYPE                 1669868 non-null  object 
 3   AMT_ANNUITY                        1297979 non-null  float32
 4   AMT_APPLICATION                    1669868 non-null  float32
 5   AMT_CREDIT                         1669867 non-null  float32
 6   AMT_DOWN_PAYMENT                   1669868 non-null  float32
 7   AMT_GOODS_PRICE                    1284699 non-null  float32
 8   WEEKDAY_APPR_PROCESS_START         1669868 non-null  object 
 9   HOUR_APPR_PROCESS_START            1669868 non-null  int8   
 10  FLAG_LAST_APPL_PER_CONTRACT        1669868 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY  

In [222]:
joined_prev.sort_values('SK_ID_CURR_x').head(10)

,SK_ID_PREV,SK_ID_CURR_x,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS_x,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,DURATION_MONTHS,MEDIAN_AMT_BALANCE,MEDIAN_AMT_CREDIT_LIMIT_ACTUAL,MEDIAN_AMT_DRAWINGS_CURRENT,MEDIAN_AMT_DRAWINGS_OTHER_CURRENT,MEDIAN_AMT_DRAWINGS_POS_CURRENT,MEDIAN_AMT_PAYMENT_TOTAL_CURRENT,MEDIAN_AMT_RECEIVABLE_PRINCIPAL,MEDIAN_AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM,MEDIAN_SK_DPD,MEDIAN_SK_DPD_DEF,max_num_instalment_version,total_instalment_number,mode_pay_ontime,mode_enough_pay
201627,1369693,100001,Consumer loans,3951.000000,24835.5,23787.0,2520.0,24835.5,FRIDAY,13,Y,1,0.104309,XAP,Approved,-1740,Cash through the bank,XAP,Family,Refreshed,Mobile,POS,XNA,Country-wide,23,Connectivity,8.0,high,365243.0,-1709.0,-1499.0,-1619.0,-1612.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,early,enough
891899,1038818,100002,Consumer loans,9251.775391,179055.0,179055.0,0.0,179055.0,SATURDAY,9,Y,1,0.000000,XAP,Approved,-606,XNA,XAP,Other,New,Vehicles,POS,XNA,Stone,500,Auto technology,24.0,low_normal,365243.0,-565.0,125.0,-25.0,-17.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,19.0,early,enough
575825,1810518,100003,Cash loans,98356.992188,900000.0,1035882.0,0.0,900000.0,FRIDAY,12,Y,1,0.000000,XNA,Approved,-746,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,low_normal,365243.0,-716.0,-386.0,-536.0,-527.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,7.0,early,enough
1223485,2396755,100003,Consumer loans,6737.310059,68809.5,68053.5,6885.0,68809.5,SATURDAY,15,Y,1,0.100037,XAP,Approved,-2341,Cash through the bank,XAP,Family,Refreshed,Consumer Electronics,POS,XNA,Country-wide,200,Consumer electronics,12.0,middle,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12.0,early,enough
1021436,2636178,100003,Consumer loans,64567.664062,337500.0,348637.5,0.0,337500.0,SUNDAY,17,Y,1,0.000000,XAP,Approved,-828,Cash through the bank,XAP,Family,Refreshed,Furniture,POS,XNA,Stone,1400,Furniture,6.0,middle,365243.0,-797.0,-647.0,-647.0,-639.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,6.0,early,enough
935358,1564014,100004,Consumer loans,5357.250000,24282.0,20106.0,4860.0,24282.0,FRIDAY,5,Y,1,0.212036,XAP,Approved,-815,Cash through the bank,XAP,Unaccompanied,New,Mobile,POS,XNA,Regional / Local,30,Connectivity,4.0,middle,365243.0,-784.0,-694.0,-724.0,-714.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,early,enough
1258847,1857999,100005,Cash loans,NaN,0.0,0.0,0.0,NaN,FRIDAY,10,Y,1,0.000000,XNA,Canceled,-315,XNA,XAP,Other,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1378692,2495675,100005,Consumer loans,4813.200195,44617.5,40153.5,4464.0,44617.5,THURSDAY,11,Y,1,0.108948,XAP,Approved,-757,Cash through the bank,XAP,Other,New,Mobile,POS,XNA,Country-wide,37,Connectivity,12.0,high,365243.0,-706.0,-376.0,-466.0,-460.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,9.0,early,enough
98145,2078043,100006,Cash loans,24246.000000,675000.0,675000.0,0.0,675000.0,THURSDAY,15,Y,1,0.000000,XNA,Approved,-181,Cash through the bank,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,48.0,low_normal,365243.0,-151.0,1259.0,-151.0,-143.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,early,enough
617735,2827850,100006,Revolving loans,NaN,0.0,0.0,0.0,NaN,THURSDAY,15,Y,1,0.000000,XA

# 4. Aggregate the joined dataset

We will first follow the aggregation in Hung's notebook. And then we will have some aggregation columns for credit card and instalment table

In [228]:
# Explicitly convert numeric columns
numeric_cols = [
    "AMT_ANNUITY", 
    "AMT_GOODS_PRICE", 
    "CNT_PAYMENT"
]

for col in numeric_cols:
    joined_prev[col] = pd.to_numeric(joined_prev[col], errors='coerce')

# Simplified aggregation dictionary
agg_dict = {
    "SK_ID_PREV": "count",
    "AMT_ANNUITY": ["median", "sum"],
    "AMT_GOODS_PRICE": ["median", "sum"],
    "CNT_PAYMENT": "median",
    "NAME_CONTRACT_TYPE": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    "NAME_CASH_LOAN_PURPOSE": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    "NAME_PAYMENT_TYPE": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    "DURATION_MONTHS": "median",
    "MEDIAN_AMT_BALANCE": "median",
    "MEDIAN_AMT_CREDIT_LIMIT_ACTUAL": "median",
    "MEDIAN_AMT_DRAWINGS_CURRENT": "median",
    "MEDIAN_AMT_DRAWINGS_OTHER_CURRENT": "median",
    "MEDIAN_AMT_DRAWINGS_POS_CURRENT": "median",
    "MEDIAN_AMT_PAYMENT_TOTAL_CURRENT": "median",
    "MEDIAN_AMT_RECEIVABLE_PRINCIPAL": "median",
    "MEDIAN_AMT_TOTAL_RECEIVABLE": "median",
    "CNT_INSTALMENT_MATURE_CUM": "median",
    "MEDIAN_SK_DPD": "median",
    "MEDIAN_SK_DPD_DEF": "median",
    "max_num_instalment_version": "median",
    "total_instalment_number": "median",
    "mode_pay_ontime": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    "mode_enough_pay": lambda x: x.mode().iloc[0] if not x.mode().empty else None
}

# Perform the aggregation
joined_prev_agg = joined_prev.groupby('SK_ID_CURR_x').agg(agg_dict).reset_index()

# If needed, you can flatten the MultiIndex columns that result from the aggregation
joined_prev_agg.columns = ['_'.join(col).strip() if type(col) is tuple else col for col in joined_prev_agg.columns]


In [229]:
joined_prev_agg.head()

,SK_ID_CURR_x_,SK_ID_PREV_count,AMT_ANNUITY_median,AMT_ANNUITY_sum,AMT_GOODS_PRICE_median,AMT_GOODS_PRICE_sum,CNT_PAYMENT_median,NAME_CONTRACT_TYPE_<lambda>,NAME_CASH_LOAN_PURPOSE_<lambda>,NAME_PAYMENT_TYPE_<lambda>,DURATION_MONTHS_median,MEDIAN_AMT_BALANCE_median,MEDIAN_AMT_CREDIT_LIMIT_ACTUAL_median,MEDIAN_AMT_DRAWINGS_CURRENT_median,MEDIAN_AMT_DRAWINGS_OTHER_CURRENT_median,MEDIAN_AMT_DRAWINGS_POS_CURRENT_median,MEDIAN_AMT_PAYMENT_TOTAL_CURRENT_median,MEDIAN_AMT_RECEIVABLE_PRINCIPAL_median,MEDIAN_AMT_TOTAL_RECEIVABLE_median,CNT_INSTALMENT_MATURE_CUM_median,MEDIAN_SK_DPD_median,MEDIAN_SK_DPD_DEF_median,max_num_instalment_version_median,total_instalment_number_median,mode_pay_ontime_<lambda>,mode_enough_pay_<lambda>
0,100001,1,3951.000000,3951.000000,24835.5,24835.5,8.0,Consumer loans,XAP,Cash through the bank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,early,enough
1,100002,1,9251.775391,9251.775391,179055.0,179055.0,24.0,Consumer loans,XAP,XNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,19.0,early,enough
2,100003,3,64567.664062,169661.968750,337500.0,1306309.5,12.0,Consumer loans,XAP,Cash through the bank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,7.0,early,enough
3,100004,1,5357.250000,5357.250000,24282.0,24282.0,4.0,Consumer loans,XAP,Cash through the bank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,early,enough
4,100005,2,4813.200195,4813.200195,44617.5,44617.5,12.0,Cash loans,XAP,Cash through the bank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,9.0,early,enough


In [232]:
joined_prev_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338857 entries, 0 to 338856
Data columns (total 26 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   SK_ID_CURR_x_                             338857 non-null  int32  
 1   SK_ID_PREV_count                          338857 non-null  int64  
 2   AMT_ANNUITY_median                        338377 non-null  float64
 3   AMT_ANNUITY_sum                           338857 non-null  float32
 4   AMT_GOODS_PRICE_median                    337793 non-null  float64
 5   AMT_GOODS_PRICE_sum                       338857 non-null  float32
 6   CNT_PAYMENT_median                        338379 non-null  float16
 7   NAME_CONTRACT_TYPE_<lambda>               338857 non-null  object 
 8   NAME_CASH_LOAN_PURPOSE_<lambda>           338857 non-null  object 
 9   NAME_PAYMENT_TYPE_<lambda>                338857 non-null  object 
 10  DURATION_MONTHS_medi

# 5. Join with downsampled application_train 

In [245]:
# Perform the left join using merge
app_w_prev_cc_instal = application_train.merge(joined_prev_agg, 
                                 left_on='SK_ID_CURR', 
                                 right_on='SK_ID_CURR_x_', 
                                 how='left')

app_w_prev_cc_instal = app_w_prev_cc_instal.drop(columns = ['SK_ID_CURR_x_'])

In [247]:
app_w_prev_cc_instal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49650 entries, 0 to 49649
Data columns (total 70 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   SK_ID_CURR                                49650 non-null  int32  
 1   TARGET                                    49650 non-null  int8   
 2   NAME_CONTRACT_TYPE                        49650 non-null  object 
 3   CODE_GENDER                               49650 non-null  object 
 4   FLAG_OWN_CAR                              49650 non-null  object 
 5   FLAG_OWN_REALTY                           49650 non-null  object 
 6   CNT_CHILDREN                              49650 non-null  float16
 7   AMT_INCOME_TOTAL                          49650 non-null  float16
 8   AMT_CREDIT                                49650 non-null  float16
 9   AMT_ANNUITY                               49650 non-null  float16
 10  AMT_GOODS_PRICE                   

Now we will remove columns with over 65% NAs.

In [250]:
# Calculate the percentage of missing values for each column
missing_percentage = app_w_prev_cc_instal.isna().mean() * 100

# Filter columns with more than 65% missing values
cols_to_drop = missing_percentage[missing_percentage > 65].index

# Drop those columns from the DataFrame
app_w_prev_cc_instal.drop(columns=cols_to_drop, inplace=True)

In [252]:
app_w_prev_cc_instal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49650 entries, 0 to 49649
Data columns (total 58 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   SK_ID_CURR                         49650 non-null  int32  
 1   TARGET                             49650 non-null  int8   
 2   NAME_CONTRACT_TYPE                 49650 non-null  object 
 3   CODE_GENDER                        49650 non-null  object 
 4   FLAG_OWN_CAR                       49650 non-null  object 
 5   FLAG_OWN_REALTY                    49650 non-null  object 
 6   CNT_CHILDREN                       49650 non-null  float16
 7   AMT_INCOME_TOTAL                   49650 non-null  float16
 8   AMT_CREDIT                         49650 non-null  float16
 9   AMT_ANNUITY                        49650 non-null  float16
 10  AMT_GOODS_PRICE                    49650 non-null  float16
 11  NAME_INCOME_TYPE                   49650 non-null  obj

In [256]:
# import file to csv
# app_w_prev_cc_instal.to_csv("app_w_prev_cc_instal.csv", index = False)